In [1]:
import chromadb

In [3]:
client = chromadb.PersistentClient(path="database/myDB")
collection = client.get_or_create_collection(name="documents")
registry = client.get_or_create_collection("corpus_model_registry")

In [5]:
collection.delete(where={"corpus_name": "Corpus 3"})

In [6]:
import os

In [7]:
model_dir = os.path.join("models")
    
if not os.path.exists(model_dir):
    print("nothing") # Return empty if no directory

# List all directories/files inside model_dir
models = [
    name for name in os.listdir(model_dir)
    if os.path.isdir(os.path.join(model_dir, name))
]
print(models)


['model1']


In [13]:
import requests

url = "http://127.0.0.1:8989/queries/model-info"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
payload = {
    "config_path": "static/config/config.yaml",
    "model_path": "models/model1"
}

response = requests.post(url, headers=headers, json=payload)

# Print response
print(response.status_code)
allInfo = response.json()


200


In [15]:
import requests

def fetch_and_process_model_info(model_path: str, endpoint: str = "http://127.0.0.1:8989/queries/model-info"):
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    payload = {
        "config_path": "static/config/config.yaml",
        "model_path": model_path
    }

    response = requests.post(endpoint, headers=headers, json=payload)
    print("Status Code:", response.status_code)
    
    if response.status_code != 200:
        raise Exception(f"Request failed with status code {response.status_code}: {response.text}")
    
    allInfo = response.json()

    themes = []
    for i, (key, val) in enumerate(allInfo.items(), start=1):
        doc_count = len(val.get("top_docs_per_topic", {}))
        themes.append({
            "id": i,
            "label": val.get("tpc_labels", f"Topic {i}"),
            "document_count": doc_count
        })

    # Sort descending by document count
    themes = sorted(themes, key=lambda t: t["document_count"], reverse=True)
    return themes


In [17]:
fetch_and_process_model_info("models/model1")

Status Code: 200


[{'id': 1, 'label': 'Topic 0', 'document_count': 3},
 {'id': 2, 'label': 'Topic 1', 'document_count': 3},
 {'id': 3, 'label': 'Topic 2', 'document_count': 3},
 {'id': 4, 'label': 'Topic 3', 'document_count': 3},
 {'id': 5, 'label': 'Topic 4', 'document_count': 3},
 {'id': 6, 'label': 'Topic 5', 'document_count': 3},
 {'id': 7, 'label': 'Topic 6', 'document_count': 3},
 {'id': 8, 'label': 'Topic 7', 'document_count': 3},
 {'id': 9, 'label': 'Topic 8', 'document_count': 3},
 {'id': 10, 'label': 'Topic 9', 'document_count': 3}]

In [14]:
import json
themes = []
for i, (key, val) in enumerate(allInfo.items(), start=1):
    doc_count = len(val.get("top_docs_per_topic", {}))
    themes.append({
        "id": i,
        "label": val.get("tpc_labels", f"Topic {i}"),
        "document_count": doc_count
    })

# Sort descending by document count
themes = sorted(themes, key=lambda t: t["document_count"], reverse=True)

# Example usage
for t in themes:
    print(t)


{'id': 1, 'label': 'Topic 0', 'document_count': 3}
{'id': 2, 'label': 'Topic 1', 'document_count': 3}
{'id': 3, 'label': 'Topic 2', 'document_count': 3}
{'id': 4, 'label': 'Topic 3', 'document_count': 3}
{'id': 5, 'label': 'Topic 4', 'document_count': 3}
{'id': 6, 'label': 'Topic 5', 'document_count': 3}
{'id': 7, 'label': 'Topic 6', 'document_count': 3}
{'id': 8, 'label': 'Topic 7', 'document_count': 3}
{'id': 9, 'label': 'Topic 8', 'document_count': 3}
{'id': 10, 'label': 'Topic 9', 'document_count': 3}


In [12]:
allInfo["t0"]

{'alphas': '17.07%',
 'tpc_labels': 'Topic 0',
 'tpc_descriptions': 'federal, state, requirements, law, information, entity, agency, action, contract, act, prohibits, order, court, person, authorized',
 'top_docs_per_topic': {'doc_newDat/newwssss.csv_194': 'employment non discrimination act prohibits employment discrimination basis actual perceived sexual orientation gender identity covered entities employers employment agencies labor organizations joint labor management committees prohibits preferential treatment quotas allows disparate treatment claims prohibits related retaliation makes act inapplicable religious organizations relationship united_states members armed_forces declares act repeal modify federal state territorial local law creating special right preference concerning employment veteran prohibits act construed prohibit employer requiring employee adhere reasonable dress grooming standards require construction new additional facilities eeoc compelling collection requiring